In [1]:
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch_geometric.loader import DataLoader
import sys
import os
import torch
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))
sys.path.append(project_root)
from gnn_model.graph_structure_from_trajecotry import node_data_list
from gnn_model.message_passing_MLP import GNN_MLP
from gnn_model.train_model import train_model

In [2]:
import torch
import random

def generate_random_positions(N, dim, min_dist, box_size):
    positions = []
    while len(positions) < N:
        pos = torch.rand(dim) * box_size
        if all(torch.norm(pos - p) >= min_dist for p in positions):
            positions.append(pos)
    return torch.stack(positions)

def generate_random_velocities(N, dim, velocity_scale=1.0):
    return (torch.rand((N, dim)) - 0.5) * 2 * velocity_scale

def compute_gravitational_forces(positions, masses, G=1.0, eps=1e-5):
    N, dim = positions.shape
    forces = torch.zeros_like(positions)
    for i in range(N):
        for j in range(i + 1, N):
            r_vec = positions[j] - positions[i]
            dist = torch.norm(r_vec) + eps
            force_mag = G * masses[i] * masses[j] / dist**2
            force_dir = r_vec / dist
            force = force_mag * force_dir
            forces[i] += force
            forces[j] -= force
    return forces

def n_body_simulation(N=5, T=100, dt=0.01, dim=2,
                      mass_range=(1.0, 10.0), min_dist=0.5,
                      box_size=10.0, velocity_scale=1.0):
    # Initialize
    masses = torch.tensor([random.uniform(*mass_range) for _ in range(N)], dtype=torch.float32)
    positions = generate_random_positions(N, dim, min_dist, box_size)
    velocities = generate_random_velocities(N, dim, velocity_scale)

    # Store results
    trajectory = torch.zeros((T, N, dim), dtype=torch.float32)
    trajectory_velocities = torch.zeros((T, N, dim), dtype=torch.float32)
    t_array  = torch.arange(0, T * dt, dt, dtype=torch.float32)

    for t in range(T):
        trajectory[t] = positions
        trajectory_velocities[t] = velocities

        # Compute forces and update positions & velocities (Euler method)
        forces = compute_gravitational_forces(positions, masses)
        accelerations = forces / masses[:, None]
        velocities = velocities + accelerations * dt
        positions = positions + velocities * dt

    trajectory_data = {
        "time": t_array,
        "positions": trajectory,
        "velocities": trajectory_velocities,
        "masses": masses
    }

    return trajectory_data

# Example usage:
trajectory_data = n_body_simulation(N=3, T=200, dt=0.01, dim=2)
# print(trajectory_data)  # (T, N, dim)

In [ ]:
nodes_data = node_data_list(trajectory_data, self_loop=False, complete_graph=True)

/Users/ruxi/Documents/GitHub/DeepLearningPH/gnn_model/graph_structure_from_trajecotry.py:81: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_target = torch.tensor(acceleration, dtype=torch.float32)


In [6]:
model = GNN_MLP(n_f=6, m_dim=2, out_channels=2, hidden_channels=128, single_node=True)

In [7]:
train_model(model, nodes_data, lr=0.005, epochs=50)

GNN_MLP()

In [8]:
# Output results
print("\nSimulation Results:")
for idx, data in enumerate(nodes_data):
    with torch.no_grad():
        pred_force = model(data.x, data.edge_index)  # Model predicts net force
        print(pred_force)
        # Ground truth force
        ground_truth_force = data.y
        print(ground_truth_force)
        
        # Ensure correct shapes and conversion to NumPy for printing
        pred_force_np = pred_force.cpu().numpy().flatten()  # Convert to NumPy array and flatten
        ground_truth_np = ground_truth_force.cpu().numpy().flatten()  # Convert to NumPy and flatten

        # Print each component of the force vector
        print(f"Sample {idx + 1}:")
        for i in range(len(pred_force_np)):
            print(f"   Component {i}: Ground Truth = {ground_truth_np[i]:.6f}, Predicted = {pred_force_np[i]:.6f}")



Simulation Results:
tensor([[-0.4380,  0.9727],
        [-0.4380,  0.9727],
        [-0.4380,  0.9727]])
tensor([[ 2.5358,  2.6829],
        [-1.6585,  1.1766],
        [-1.4329, -3.0288]])
Sample 1:
   Component 0: Ground Truth = 2.535820, Predicted = -0.438032
   Component 1: Ground Truth = 2.682906, Predicted = 0.972715
   Component 2: Ground Truth = -1.658547, Predicted = -0.438032
   Component 3: Ground Truth = 1.176596, Predicted = 0.972715
   Component 4: Ground Truth = -1.432860, Predicted = -0.438032
   Component 5: Ground Truth = -3.028830, Predicted = 0.972715
tensor([[-0.4380,  0.9727],
        [-0.4380,  0.9727],
        [-0.4380,  0.9727]])
tensor([[ 2.5607,  2.7408],
        [-1.6550,  1.1742],
        [-1.4571, -3.0798]])
Sample 2:
   Component 0: Ground Truth = 2.560723, Predicted = -0.438032
   Component 1: Ground Truth = 2.740794, Predicted = 0.972715
   Component 2: Ground Truth = -1.655048, Predicted = -0.438032
   Component 3: Ground Truth = 1.174212, Predicted =